In [1]:
from IPython.display import HTML
from pathlib import Path

css_rules = Path('../custom.css').read_text()
HTML('<style>' + css_rules + '</style>')

# Ficheros, módulos y paquetes

![Save](img/save.png)

En esta sección veremos cómo trabajar con *ficheros* en Python, y también cómo organizar nuestro código en *módulos* y *paquetes*, un nivel superior de jerarquía para estructurar mejor nuestros programas.

> Icons made by <a href="https://www.flaticon.com/authors/smashicons" title="Smashicons">Smashicons</a> from <a href="https://www.flaticon.com/" title="Flaticon"> www.flaticon.com</a>

## 📁 Ficheros

Un **fichero** es una secuencia de bytes almacenados en algún *sistema de ficheros* y accesibles por un *nombre de fichero*. Un *directorio* (o *carpeta*) es una colección de ficheros, y probablemente de otros directorios.

Muchos sistemas de ficheros son jerárquicos y a menudo nos referimos a ellos como *árbol de ficheros*.

### Crear o abrir un fichero

Python ofrece la función `open()` para realizar las siguientes acciones:
- Leer contenido un fichero existente.
- Escribir contenido en un nuevo fichero.
- Añadir contenido a un fichero existente.

![Open files](img/open-files.png)

In [2]:
fout = open('oops.txt', 'wt')

In [3]:
!ls -l oops.txt

-rw-r--r--@ 1 sdelquin  staff  0  8 abr 13:39 oops.txt


In [4]:
type(fout)

_io.TextIOWrapper

> Podemos ver que el manejador (handler) del fichero es un [buffer de texto](https://docs.python.org/3/library/io.html#io.TextIOWrapper).

Después de abrir un fichero necesitamos cerrarlo para asegurarnos que todas las operaciones de escritura pendientes (*buffers*) son ejecutadas. También permite liberar la memoria asociada al recurso.

In [5]:
fout.close()

### Rutas relativas y absolutas

Cuando tenemos que especificar la ruta a un fichero, ésta se puede indicar en forma relativa o absoluta:

- Ruta **relativa**: es aquella que toma como referencia la carpeta actual de trabajo.
- Ruta **absoluta**: es aquella que toma como referencia la raíz del sistema de ficheros.

![Paths](img/paths.png)

> En negro se muestra *rutas absolutas* y en rojo las *rutas relativas* (desde el punto en el que se encuentran).

### Escribir en un fichero de texto con redirección

La función `print()` dispone de un argumento para indicar a qué "*dispositivo"* redirigir la salida:

In [6]:
fout = open('oops.txt', 'w')  # equivalente a 'wt'
print('Oops, I created a file.', file=fout)
fout.close()

In [7]:
!cat oops.txt

Oops, I created a file.


### Escribir en un fichero de texto

El método `write()` nos permite escribir texto en un fichero:

In [8]:
msg = 'Oops, I am writing text through another method.'
len(msg)

47

In [9]:
fout = open('oops.txt', 'w')
fout.write(msg)  # devuelve el número de bytes escritos

47

In [10]:
fout.close()

In [11]:
!cat oops.txt

Oops, I am writing text through another method.

> Hay que tener en cuenta que el método `write()` no añade salto de línea `\n`. Si se quiere, hay que indicarlo explícitamente.

### Leer un fichero de texto de una vez

En primer lugar vamos a escribir un texto multilínea en un fichero para luego utilizarlo en los métodos de lectura:

In [12]:
lyrics = '''The lights go out and I can't be saved
Tides that I tried to swim against
Have brought me down upon my knees
Oh I beg, I beg and plead, singing'''  # Clocks by Coldplay

In [13]:
fout = open('oops.txt', 'w')
fout.write(lyrics)
fout.close()

En este caso usamos el método `read()`:

In [14]:
fin = open('oops.txt', 'rt')
loaded_lyrics = fin.read()
fin.close()

In [15]:
loaded_lyrics == lyrics  # coincide con los datos escritos

True

### `FileNotFoundError`

Si vamos a abrir para lectura un fichero que no existe en el sistema de ficheros, obtendremos una excepción de tipo `FileNotFoundError`:

In [16]:
fin = open('foo.txt', 'rt')

FileNotFoundError: [Errno 2] No such file or directory: 'foo.txt'

Podemos manejar este error capturando la excepción asociada:

In [17]:
try:
    fin = open('foo.txt', 'rt')
except FileNotFoundError as err:
    print('⛔️ Check the path of the file!')
    print(err)

⛔️ Check the path of the file!
[Errno 2] No such file or directory: 'foo.txt'


### Leer un fichero de texto línea a línea

Python ofrece el método `readline()` para leer de un fichero de texto únicamente *una línea de cada vez*:

In [18]:
lyrics = ''

fin = open('oops.txt', 'r')  # equivalente a 'rt'

while True:
    line = fin.readline()  # si no hay más contenido devuelve cadena vacía ''
    if not line:
        break
    lyrics += line

fin.close()

In [19]:
print(lyrics)

The lights go out and I can't be saved
Tides that I tried to swim against
Have brought me down upon my knees
Oh I beg, I beg and plead, singing


### Leer un fichero de texto línea a línea (con iterador)

El manejador (*handler*) del fichero es en sí mismo un *iterable*, con lo cual podemos recorrerlo de la manera habitual:

In [20]:
fin = open('oops.txt', 'r')

for line in fin:
    print(repr(line))  # repr() muestra caracteres de control

fin.close()

"The lights go out and I can't be saved\n"
'Tides that I tried to swim against\n'
'Have brought me down upon my knees\n'
'Oh I beg, I beg and plead, singing'


> OJO! Las líneas que leemos del fichero contienen los saltos de línea `\n`

### Leer un fichero de texto como líneas separadas

Existe una forma de leer un fichero de texto como una **lista de cadenas de texto** (*cada elemento de la lista es una línea del fichero*) utilizando el método `readlines()`:

In [21]:
fin = open('oops.txt')  # equivalente a 'r' == 'rt'
lines = fin.readlines()
fin.close()

In [22]:
print(len(lines), 'lines read')

4 lines read


In [23]:
for line in lines:
    print(line, end='')  # estamos quitando el salto de línea extra de "print"

The lights go out and I can't be saved
Tides that I tried to swim against
Have brought me down upon my knees
Oh I beg, I beg and plead, singing

> Cuidado con este mecanismo cuando se están manejando ficheros muy grandes ya que todo el contenido del fichero se carga de una sola vez en memoria principal.

### Cerrar automáticamente ficheros

Python ofrece los [gestores de contexto](https://docs.python.org/3/reference/datamodel.html#context-managers) que permiten establecer reglas de entrada y salida al contexto definido. En el caso que nos ocupa usaremos la sentencia `with` y el contexto se ocupará de cerrar adecuadamente el fichero que hemos abierto, liberando así sus recursos:

In [24]:
with open('oops.txt', 'w') as fout:
    fout.write('Writing within a context manager!')

In [25]:
!cat oops.txt

Writing within a context manager!

### Forma "canónica" de leer un fichero (línea a línea)

Con todo lo que hemos visto, la forma "canónica" (me atrevería a decir *pitónica*) de recorrer un fichero línea a línea sería la siguiente:

In [26]:
with open('oops.txt') as f:
    for line in f:
        data = line.strip()  # remove blank chars (including linebreaks)
        # process "data" in any way
        print(data)

Writing within a context manager!


### 🎯 Ejercicio

Dado el fichero [temperatures.txt](./files/temperatures.txt) con 12 líneas (*meses*) y temperaturas de cada día, se pide:

1. Leer el fichero de datos.
2. Calcular la temperatura media de cada mes.
3. Escribir un fichero de salida `avgtemps.txt` con 12 líneas (*meses*) y la temperatura media de cada mes.

&nbsp;

> Para saber en qué carpeta está trabajando puede usar estos comandos desde Jupyter Notebook:
- `!echo %cd%` (*Windows*)
- `!pwd` (*Linux, Mac*)

<hr>

**📎 Posible solución:** [solutions/avgtemps.py](solutions/avgtemps.py)

In [27]:
# Escriba aquí su solución

In [28]:
# %load "solutions/avgtemps.py"

## 🧩 Módulos

Es una certeza que, más pronto que tarde, usaremos código Python en más de un fichero. Un **módulo** es simplemente un fichero con código Python. No se necesita hacer nada especial. Cualquier código Python se puede usar como un módulo por otros.

Para hacer uso del código de otros módulos usaremos la sentencia `import`. Esto permite importar el código y las variables de dicho módulo para que estén disponibles en tu programa.

### Importar un módulo

La forma más sencilla de importar un módulo es `import <module>` donde *module* es el nombre de otro fichero Python, sin la extensión `.py`.

Veamos un breve ejemplo haciendo uso de la librería de *valores aleatorios* de Python:

In [ ]:
# %load "files/fast.py"
from random import choice

places = [
    'McDonalds', 'KFC', 'Burger King', 'Taco Bell', 'Wendys', 'Arbys',
    'Pizza Hut'
]


def pick():
    ''' Return random fast food place'''
    return choice(places)


In [ ]:
# %load "files/lunch.py"
import fast

place = fast.pick()
print("Let's go to", place)


Ejecución del programa principal:

In [31]:
%run "files/lunch.py"

Let's go to McDonalds


In [32]:
%run "files/lunch.py"

Let's go to McDonalds


In [33]:
%run "files/lunch.py"

Let's go to Arbys


### Reglas de estilo al importar módulos

La [guía de estilo de Python (PEP8)](https://www.python.org/dev/peps/pep-0008/#imports) establece una serie de recomendaciones a la hora de importar módulos, entre las que destaco:

1. Importar módulos en distintas líneas.
2. Importar módulos al principio del fichero con un orden:
    1. Librería estándar.
    2. Librerías de terceros.
    3. Librerías propias.
3. Evitar importar con comodines (`*`)

### Importar un módulo con otro nombre

Por cuestiones de legibilidad o de colisión de nombres, es posible que queramos importar un módulo con un nombre diferente al que tiene. Para ello podemos utilizar la sentencia `as` indicando el nombre deseado:

In [ ]:
# %load "files/fast2.py"
import fast as f

place = f.pick()
print("Let's go to", place)


In [35]:
%run "files/fast2.py"

Let's go to KFC


In [36]:
%run "files/fast2.py"

Let's go to KFC


### Importa sólo lo que necesites

Tenemos la posibilidad de importar un módulo completo o bien sólo partes de él.  Veamos un ejemplo:

In [ ]:
# %load "files/fast3.py"
from fast import pick

place = pick()
print("Let's go to", place)


In [38]:
%run "files/fast3.py"

Let's go to Arbys


In [39]:
%run "files/fast3.py"

Let's go to Arbys


## 📦 Paquetes

Un **paquete** es simplemente un subdirectorio que contiene ficheros `.py`. Permite tener jerarquía con más de un nivel de directorios anidados.

Vamos a crear un paquete `choices` ampliando el ejemplo anterior:

![Package](img/package.png)

In [ ]:
# %load "files/choices/fast.py"
from random import choice

places = [
    'McDonalds', 'KFC', 'Burger King', 'Taco Bell', 'Wendys', 'Arbys',
    'Pizza Hut'
]


def pick():
    ''' Return random fast food place'''
    return choice(places)


In [ ]:
# %load "files/choices/advice.py"
from random import choice

answers = ['Yes!', 'No!', 'Reply hazy', 'Sorry, what?']


def give():
    '''Return random advice'''
    return choice(answers)


In [ ]:
# %load "files/questions.py"
from choices import fast, advice

print("Let's go to", fast.pick())
print('Should we take out?', advice.give())


Veamos la ejecución del programa principal:

In [43]:
%run "files/questions.py"

Let's go to Arbys
Should we take out? Sorry, what?


In [44]:
%run "files/questions.py"

Let's go to Taco Bell
Should we take out? Yes!


In [45]:
%run "files/questions.py"

Let's go to KFC
Should we take out? Sorry, what?


> En versiones anteriores a Python 3.3 era necesario incluir un fichero en blanco llamado `__init__.py` dentro del directorio para que el paquete fuera reconocible.

### La ruta de búsqueda de módulos

En el ejemplo anterior hemos visto que Python busca en el directorio actual por paquetes a los que hagamos referencia en nuestro código. Pero existen otras rutas que usa para ello. De hecho los módulos de la librería estándar no están en nuestro directorio de trabajo pero sí podemos usarlas.

Veamos cómo acceder, e incluso modificar, las ruta de búsqueda de módulos:

In [46]:
import sys

for place in sys.path:
    print(place)

/Users/sdelquin/Dropbox/Code/pythoncanarias/eoi/02-core/10-files
/Users/sdelquin/.virtualenvs/eoi/lib/python38.zip
/Users/sdelquin/.virtualenvs/eoi/lib/python3.8
/Users/sdelquin/.virtualenvs/eoi/lib/python3.8/lib-dynload
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8

/Users/sdelquin/.virtualenvs/eoi/lib/python3.8/site-packages
/Users/sdelquin/.virtualenvs/eoi/lib/python3.8/site-packages/IPython/extensions
/Users/sdelquin/.ipython


> La línea en blanco hace referencia al directorio actual.

El *orden* de `sys.path` es importante, ya que Python buscará el módulo referenciado empezando por el primer elemento de esta lista. Esto significa que si queremos importar un módulo `lucky` que, por casualidad, ya está definido en alguna ruta previa, no podremos usarlo.

Sin embargo es posible modificar la ruta de búsqueda desde nuestro código. Supongamos que queremos que Python busque en la ruta `/home/sergio/dev` antes que en ninguna otra:

~~~python
>>> import sys

>>> sys.path.insert(0, '/home/sergio/dev')
~~~

### Importaciones absolutas y relativas

Python soporta importaciones *absolutas* y *relativas*. Hasta ahora todos los ejemplos que hemos visto han usado importaciones absolutas. Si escribimos `import basics` Python buscará un fichero llamado `basics.py` (*módulo*) o un directorio llamado `basics` (*paquete*).

Pero podemos utilizar la notación `.` y `..` para hacer referencia a ubicaciones (*rutas*) relativas a nuestro código actual:

- Si se encuentra en el directorio actual:  
`from . import basics`
- Si se encuentra en el directorio superior:  
`from .. import basics`
- Si se encuentra en un directorio hermano del superior:  
`from ..utils import basics`

> La notación `.` y `..` proviene de los atajos Unix con los que se representaban el directorio actual y el directorio superior.

## 🔶 Programa principal

Cuando se pide hacer un programa en Python, solemos tener un programa principal que contiene el **punto de entrada** de la ejecución. A partir de ahí se puede hacer uso de recursos en ese mismo fichero o en cualquier otra librería (paquete) que tengamos a nuestra disposición.

~~~python
# stdlib imports
# third party imports (pip install ...)
# custom imports

class ClassA:
    ...

class ClassB:
    ...

def func_a():
    ...

def func_b():
    ...

if __name__ == '__main__':
    obj1 = ClassA()
    obj2 = ClassB()
    aux = obj1 + obj2
    result = func_a(obj1) + func_b(obj2)
    print(result)
~~~

### `if __name__ == '__main__'`

Esta condición permite, en el programa principal, diferenciar qué codigo se lanzará cuando el fichero se ejecuta directamente o cuando el fichero se importa desde otro código:

![if-name-main](img/if-name-main.png)

### 🎯 Ejercicio

Escriba un programa que lea un número variable de enteros por línea de comandos y devuelva los siguientes cálculos: *suma, máximo, mínimo, media y desviación típica*.

![xmath](img/xmath.png)

> Cree los ficheros tal y como se indican en la figura, haciendo las importaciones necesarias.

<hr>

**📎 Posible solución:** [solutions/main.py](solutions/main.py) | [solutions/xmath.py](solutions/xmath.py)

In [47]:
# Escriba aquí su solución

In [48]:
# %load "solutions/main.py"

In [49]:
# %load "solutions/xmath.py"